### Concate train and test files.

In [1]:
import pandas as pd, numpy as np
import pickle
from sklearn.model_selection import train_test_split


In [13]:
# Combine all TRAC test and gold data. Whoops...


trac_fb = pd.read_csv('../Data/TRAC2018_full/converted/trac-gold-set/agr_en_fb_gold.csv')
trac_sm = pd.read_csv('../Data/TRAC2018_full/converted/trac-gold-set/agr_en_tw_gold.csv')

frames = [trac_fb, trac_sm]
df_trac_test_combined = pd.concat(frames)


df_trac_test_combined.to_csv('../Data/TRAC2018_full/converted/trac-gold-set/agr_en_combined_gold.csv', index=None)


In [34]:
#pd.read_csv('../Data/gibert_vua_format/trainData.csv', sep='\t')
pd.read_csv('../Data/hate_speech_mlma/MLMA_convert/level1/test.tsv', sep='\t', usecols=['text','label'])

ValueError: Usecols do not match columns, columns expected but not found: ['label']

In [2]:

def has_duplicated(l):
    return pd.Series(l).duplicated().sum() > 0

print(has_duplicated(['one', 'two', 'one']))
# True
print(has_duplicated(['one', 'two', 'three']))
# False

True
False


In [26]:
def pickle_data(train_file, test_file, type='csv'):
    '''Concat and convert VUA data to pickle files for Deepmoji.

    Args: 
        train_file (str): filepath to a training datafile.
        test_file (str): filepath to a test_file.
    
    Optional:
        type (str): specify data file format (default is .csv)

    '''

    # 1. Load the data

    cols_to_use = ['text', 'label']
    df_train = pd.read_csv(train_file, usecols=cols_to_use)
    df_test = pd.read_csv(test_file, usecols=cols_to_use)

    #print(len(df_train), len(df_test))

    # Concat'em
    frames = [df_train, df_test]
    df_train_and_test = pd.concat(frames)

    # Rename columns to deepmoji's format.
    df_train_and_test.rename({'text': 'texts', 'label':'info'}, axis=1, inplace=True)
    
    # Modify info column values 'str' -> {'label':label}
    df_train_and_test['info'] = df_train_and_test['info'].apply(lambda x: {'label' : x})
    
    # Add the extra index columns
    extra_index_cols =  ['train_ind', 'test_ind', 'val_ind']
    df_train_and_test = df_train_and_test.join(pd.DataFrame(columns=extra_index_cols)) #.fillna(np.nan)
    

    # 2. Construct dict for pickeling.
    # Get indeces

    # Split original train data into train and validation sets.
    org_train_ind = (range(0, len(df_train)))

    train_val_split = train_test_split(org_train_ind, test_size=0.1)
    train_ind = train_val_split[0]
    val_ind = train_val_split[1]

    # Add new train indeces.
    #df_train_and_test['train_ind'] = df_train_and_test['train_ind'].add(train_ind, ignore_index=True)
    #df_train_and_test['train_ind'] = pd.Series(train_ind) # this used to work..
    df_train_and_test.iloc[:len(train_ind):, 2] = train_ind

    # Add val indeces
    #df_train_and_test['val_ind'] = pd.Series(val_ind) # this used to work.
    df_train_and_test.iloc[:len(val_ind):, 4] = val_ind

    # Add test indeces
    test_ind = range(len(train_ind), len(df_train_and_test))
    #df_train_and_test.loc[len(train_ind), 'test_ind'] = test_ind
    df_train_and_test.iloc[len(train_ind):, 3] =  test_ind # this works
    #df_train_and_test.insert(loc=len(train_ind), column='test_ind', value=test_ind, allow_duplicates=False)
    

    # Check duplicates
    #dupes = df_train_and_test.duplicated(subset=['train_ind'])

    # 3. Pickle the dict.


    output = df_train_and_test.to_dict('list')
    
    with open('./mlma_lvl_1_data.pkl', 'wb') as handle:
       pickle.dump(output , handle)

    # This is just for testing.
    return output




In [31]:
# Function call

# TRAC
test = pickle_data('../Data/hate_speech_mlma/MLMA_convert/level1/train.tsv' , '../Data/hate_speech_mlma/MLMA_convert/level1/test.tsv')

ValueError: Usecols do not match columns, columns expected but not found: ['text', 'label']

In [28]:
#test.keys()
#test['texts'][:10]

#print(' length', len(test['texts']))

# Check if things align. (✔)

#print(test['texts'][-1])
#print(test['info'][-1:])
#print(test['train_ind'][-1:])

print(len(test['train_ind']))

df_test = pd.DataFrame(test)
df_test


2392


,texts,info,train_ind,test_ind,val_ind
0,Ready .... GO !!!,{'label': 'noHate'},1616.0,NaN,1532.0
1,If i sent you some pic wound use be able to pu...,{'label': 'noHate'},1678.0,NaN,1792.0
2,ALWAYS SPEAK OUT WHEN IN A CLASS WITH A BRAINW...,{'label': 'noHate'},592.0,NaN,642.0
3,They fit the same DNA as the DNA found in anci...,{'label': 'noHate'},1224.0,NaN,181.0
4,I would not be surprised if this woman was spe...,{'label': 'hate'},395.0,NaN,795.0
...,...,...,...,...,...
2387,Sometimes I do watch shows about WWII on the H...,{'label': 'noHate'},NaN,2387.0,NaN
2388,It 's a pretty powerful documentary that blew ...,{'label': 'noHate'},NaN,2388.0,NaN
2389,Like the parasites they are they simply moved ...,{'label': 'hate'},NaN,2389.0,NaN
2390,Their lies are so thick and many that people d...,{'label': 'hate'},NaN,2390.0,NaN


In [8]:
# inspect pickle file

with open('./trac_data.pkl', 'rb') as handle:
    data = pickle.load(handle)

print(len(data), type(data), data.keys())

print(len(data['texts']))

5 <class 'dict'> dict_keys(['texts', 'info', 'train_ind', 'test_ind', 'val_ind'])
14172
